In [1]:
import sys
import pandas as pd 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet
from quickbin import bin2d

from photutils.background import Background2D, MedianBackground
from astropy.stats import SigmaClip

In [ ]:
def centile_clip(image, centiles=(1, 99)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
# make a list of fuv to make / look at 

og_aspect = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet').to_pandas()
og_aspect = og_aspect.dropna(subset=['fuv_temp'])


In [ ]:
fuv = og_aspect.iloc[400:500]

In [ ]:
fuv

In [ ]:
for index, eclipse in fuv['eclipse'].items():
    print(eclipse)
    try: 
        execute_pipeline(
                int(eclipse),
                "FUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/media/bekah/BekahA/background_test/pics_and_photonlists",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": False},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        
    except Exception as e: 
        print(e)


In [ ]:
for index, eclipse in fuv['eclipse'].items():
    
    try: 
        ecl = fits.open(f"/media/bekah/BekahA/background_test/e{eclipse}_FUV_threshold_nobkgsub.fits")

        cnt_image = ecl[1].data
        sigma_clip = SigmaClip(sigma=3.)
        bkg_estimator = MedianBackground()
        bkg = Background2D(cnt_image,(50, 50),filter_size=(3, 3), 
                           bkg_estimator=bkg_estimator,
                               sigma_clip=sigma_clip)

        fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(10, 8))

        ax1.imshow(centile_clip(bkg.background), cmap='grey',interpolation=None)
        ax1.set_title('background')
        ax1.axis('off')

        ax2.imshow(centile_clip(np.multiply(3,bkg.background_rms)) , cmap='grey',interpolation=None)
        ax2.set_title('threshold')
        ax2.axis('off')  # Hide axes

        plt.tight_layout()

        plt.savefig(f'/media/bekah/BekahA/background_test/pics_and_photonlists/e{eclipse}_fuv_bkg.png')

    except Exception as e: 
        print(e)


In [ ]:
for index, eclipse in fuv['eclipse'].items():
    
    try: 
        ecl = fits.open(f"/media/bekah/BekahA/background_test/e{eclipse}_FUV_threshold_nobkgsub.fits")
        
        data = ecl[1].data


        # lowest non-zero value
        non_zero_min = np.min(data[data > 0])

        # replace all zero / NaN values with the lowest non-zero value
        data[data == 0] = non_zero_min
        data = np.nan_to_num(data, nan=non_zero_min)  

        sigma_clip = SigmaClip(sigma=3.)
        bkg_estimator = MedianBackground()
        bkg = Background2D(data,(50, 50),filter_size=(3, 3), 
                           bkg_estimator=bkg_estimator,
                               sigma_clip=sigma_clip)

        fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(10, 8))

        ax1.imshow(centile_clip(bkg.background), cmap='grey',interpolation=None)
        ax1.set_title('background')
        ax1.axis('off')

        ax2.imshow(centile_clip(np.multiply(3,bkg.background_rms)) , cmap='grey',interpolation=None)
        ax2.set_title('threshold')
        ax2.axis('off')  # Hide axes

        plt.tight_layout()

        plt.savefig(f'/media/bekah/BekahA/background_test/pics_and_photonlists/mod_e{eclipse}_fuv_bkg.png')
        
    except Exception as e: 
        print(e)

In [ ]:
# flat scale parameter 
flat_correct = -0.0031
flat_t0 = 840418779.02
flat_correct_0 = 1.2420282
flat_correct_1 = -2.8843099e-10
flat_correct_2 = 0.000

t = np.linspace(flat_t0,flat_t0+10000000, 100000)

flat_scale = (
        flat_correct_0 + (flat_correct_1 * t) + (flat_correct_2 * t * t))

plt.plot(t, flat_scale)

plt.title("fuv flat scale")
plt.xlabel("time")

In [ ]:
flat_correct = -0.0154
flat_t0 = 840418779.02
flat_correct_0 = 1.9946352
flat_correct_1 = -1.9679445e-09
flat_correct_2 = 9.3025231e-19

t = np.linspace(flat_t0, flat_t0+10000000, 100000)

flat_scale = (
        flat_correct_0 + (flat_correct_1 * t) + (flat_correct_2 * t) * t
    )

plt.plot(t, flat_scale)

plt.title("nuv flat scale")
plt.xlabel("time")

In [ ]:
flat_correct_fuv=-0.0031,
flat_t0_fuv=840418779.02, 

flat_correct_nuv=-0.0154,
flat_t0_nuv=840418779.02, 

flat_correct_fuv_0=1.2420282, 
flat_correct_fuv_1=-2.8843099e-10, 
flat_correct_fuv_2=0.000, 

flat_correct_nuv_0=1.9946352, 
flat_correct_nuv_1=-1.9679445e-09, 
flat_correct_nuv_2=9.3025231e-19

In [ ]:
"cal_info,calinf,cali,cal2_info,cal2: list of double  = 
{dumwig=>0,ne=>0,flat_t0_nuv=>840418779.02,flat_t0_fuv=>840418779.02,linfile_fuv_x=>FUV_NLC_x_det2sky.fits,pltscl=>68.754932,maskfile_fuv=>FUV_mask.fits,
 flat_correct_nuv=>-0.0154,wigfile_fuv_x=>FUV_wiggle_x.fits,
 spflatfile=>spflat.fits,nb=>1,fe=>0,fc=>0,wigfile_nuv_x=>NUV_wiggle_x.fits,nc=>0,
 walkfile_fuv_x=>FUV_walk_x.fits,ff=>1,nd=>0,flatfile=>flat.fits,walkfile_nuv_y=>NUV_walk_y.fits,
 detbgfile=>detbg.fits,linfile_nuv_x=>NUV_NLC_x_det2sky.fits,detsize=>1.25,nf=>1,
 wigfile_nuv_y=>NUV_wiggle_y.fits,linfile_nuv_y=>NUV_NLC_y_det2sky.fits,
 flat_correct_nuv_2=>9.3025231e-19,fb=>1,maskfile_nuv=>NUV_mask.fits,
 flat_correct_fuv_2=>0.000,walkfile_fuv_y=>FUV_walk_y.fits,maskfile=>mask.fits,
 fd=>0,wigfile_fuv_y=>FUV_wiggle_y.fits,flat_correct_fuv_1=>-2.8843099e-10,
 linfile_fuv_y=>FUV_NLC_y_det2sky.fits,dumlin=>0,dummy=>0                           
 ,fa=>0,calpath=>/home/galex/cal/targ/use,dumwalk=>0,walkfile_nuv_x=>NUV_walk_x.fits,
 maskr=>0,flat_correct_fuv=>-0.0031,flat_correct_fuv_0=>1.2420282,
 fflat_correct_nuv_1=>-1.9679445e-09,flat_correct_nuv_0=>1.9946352,na=>0} # Some calibration info\n"


In [ ]:
from gPhoton.io.mast import get_raw_paths
eclipse = 11000
urls = get_raw_paths(eclipse)
urls

In [2]:
scst = fits.open("/home/bekah/Downloads/e11000-scst.fits")
scst

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f22e5f1a810>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f22e5f351d0>]

In [ ]:
import pyarrow.parquet as pq 

In [ ]:
file_path = '/home/bekah/Downloads/extended_scst_7_16 (1).parquet'

parquet_file = pq.ParquetFile(file_path)

print("Columns and types:")
print(parquet_file.schema)

table = pq.read_table(file_path, columns=['eclipse', 'hvnom_fuv']).to_pandas()


In [ ]:
table

In [ ]:
plt.scatter(table['eclipse'],table['hvnom_fuv'],s=.2)

In [8]:
scst[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT = '        '           /   FITS (Flexible Image Transport System) format
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
GRELEASE= 'ops-v7_2_0'                                                          
TRANGE0 = '050520T223016Z'                                                      
TRANGE1 = '050521T000851Z'                                                      
RA_CENT =            239.14792                                                  
DEC_CENT=              10.8725                                                  
MPSTYPE = 'GIS     '                                                            
MPSPLAN = 'gip     '        

In [ ]:
# ok which of these do I want? 

FDTTDC  =     30.2463488799476  # FUV temps                                                                                              
ITFUVOBA=     23.6954584662164   # Instrument Temps B                                                                                             
FDHVMON =     6212.70507858441   # Voltages B or Detector HV?                                              
FDHIMON =     48.5158041971777   # Detector HV                                                
FDSFHV  =     10.2447352131722   # fuv_normal_operation_indicator_for_hv                                               
FDCTEC  =     1512.71322743765   # FUV count rate                                                
FDCFEC  =      1665.3077821901   # FUV count rate                                                
TRANGE0 = '050304T113547Z'                                                      
TRANGE1 = '050304T131424Z'  

    ['FDTTDC','ITFUVOBA','FDHVMON','FDHIMON','FDSFHV','FDCTEC','FDCFEC','TRANGE0','TRANGE1']